In [1]:
import pandas as pd
import re
import string
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np

from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.under_sampling import RandomUnderSampler



import pickle


In [3]:
df = pd.read_json('Dataset/test.json')
df.head()

,name,gender,birth_date,interest,searchs
0,Phoebe.Satterfield,female,2022-10-06T05:04:50.87+08:00,"[Beach, Museum, Peak, Mountain, Lake, Waterfall]","[{'count': 23, 'name': 'Taman Nasional Gunung ..."
1,Phoebe.Satterfield,female,2022-10-06T05:04:50.87+08:00,"[Beach, Museum, Peak, Mountain, Lake, Waterfall]","[{'count': 23, 'name': 'Taman Nasional Gunung ..."
2,Phoebe.Satterfield,female,2022-10-06T05:04:50.87+08:00,"[Beach, Museum, Peak, Mountain, Lake, Waterfall]","[{'count': 23, 'name': 'Taman Nasional Gunung ..."
3,Phoebe.Satterfield,female,2022-10-06T05:04:50.87+08:00,"[Beach, Museum, Peak, Mountain, Lake, Waterfall]","[{'count': 23, 'name': 'Taman Nasional Gunung ..."
4,Phoebe.Satterfield,female,2022-10-06T05:04:50.87+08:00,"[Beach, Museum, Peak, Mountain, Lake, Waterfall]","[{'count': 23, 'name': 'Taman Nasional Gunung ..."


In [11]:
import json
import pandas as pd
from datetime import datetime

# === Load JSON ===
with open("Dataset/test.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# === Fungsi untuk menghitung umur dari birth_date ===
def calculate_age(birth_date_str):
    if not birth_date_str:
        return None
    try:
        birth_date = datetime.fromisoformat(birth_date_str.replace("Z", "+00:00"))
        today = datetime.today()
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    except Exception as e:
        print(f"Error parsing date '{birth_date_str}': {e}")
        return None

# === Proses data baris demi baris ===
processed_rows = []

for user in data:
    user_id = user.get("name", "unknown")
    gender = user.get("gender", "unknown")
    birth_date = user.get("birth_date", None)
    age = calculate_age(birth_date)

    # Proses interest (pastikan semua elemen adalah string)
    interests_raw = user.get("interest", [])
    interests = ", ".join([str(i) for i in interests_raw if i]) if interests_raw else ""

    # Proses data pencarian
    searchs = user.get("searchs")
    if isinstance(searchs, list):
        for search in searchs:
            destination = search.get("name", "unknown")
            search_count = search.get("count", 0)
    
            processed_rows.append({
                "user_id": user_id,
                "age": age,
                "gender": gender,
                "interests": interests,
                "destination": destination,
                "search_count": search_count
            })

        
# === Simpan ke DataFrame dan CSV ===
df = pd.DataFrame(processed_rows)

# Simpan ke file CSV (untuk pelatihan model ML)
df.to_csv("Dataset/processed_user_data.csv", index=False)

# Tampilkan contoh data
print("✅ Data berhasil diproses!")
print(df.head(10))  # Tampilkan 10 baris pertama


✅ Data berhasil diproses!
              user_id  age  gender  \
0  Phoebe.Satterfield    2  female   
1  Phoebe.Satterfield    2  female   
2  Phoebe.Satterfield    2  female   
3  Phoebe.Satterfield    2  female   
4  Phoebe.Satterfield    2  female   
5  Phoebe.Satterfield    2  female   
6  Phoebe.Satterfield    2  female   
7  Phoebe.Satterfield    2  female   
8  Phoebe.Satterfield    2  female   
9  Phoebe.Satterfield    2  female   

                                        interests  \
0  Beach, Museum, Peak, Mountain, Lake, Waterfall   
1  Beach, Museum, Peak, Mountain, Lake, Waterfall   
2  Beach, Museum, Peak, Mountain, Lake, Waterfall   
3  Beach, Museum, Peak, Mountain, Lake, Waterfall   
4  Beach, Museum, Peak, Mountain, Lake, Waterfall   
5  Beach, Museum, Peak, Mountain, Lake, Waterfall   
6  Beach, Museum, Peak, Mountain, Lake, Waterfall   
7  Beach, Museum, Peak, Mountain, Lake, Waterfall   
8  Beach, Museum, Peak, Mountain, Lake, Waterfall   
9  Beach, Museum, Peak, M